# NUSTAR SCIENTIFIC PRODUCTS EXTRACTION

In [2]:
from python_for_nustar.nu_core import np, plt, os, glob, scan_phase_resolved_products, reduce_list
from python_for_nustar.nu_class import NustarObservation
from python_for_nustar.pipeline.sj0243_init import Nu_path, ObsList_bright, ObsList, periods_val


## Running nupipeline and  source and background  regions extraction ds9 scripts

In [4]:
#######   nupipeline and regions   #######
for ObsID_tmp in ObsList:
    nu_obs = NustarObservation(ObsID_tmp, nu_path=Nu_path)
    nu_obs.nupipeline(ObsList_bright=ObsList_bright)
    nu_obs.make_regions()

###
Observation 90302319002 loaded successfully


Exception: nupipeline has already been launched!

## Initialize  NustarObservation object of particular ObsID

In [ ]:
ObsID =  ObsList[0]
nu_obs = NustarObservation(ObsID, nu_path=Nu_path, orb_params=orb_params_pulsar)
open_dir_in_term()

## basic nuproducts:
extract spectra and light curve from both modules. 
gorup sepectra, make barycentric correction of light curves 

In [ ]:
prodpath_ave = 'spe_and_lc' #folder name for basivc products
for mode in ['A', 'B']:      
    nu_obs.nuproducts(outdir=prodpath_ave, mode=mode, stemout=prodpath_ave ,binsize='0.01')


lclist = [f'{prodpath_ave}{mode}_sr.lc' for mode in ['A', 'B']]
spelist = [f'{prodpath_ave}{mode}_sr.pha' for mode in ['A', 'B']]

nu_obs.grppha(infiles=spelist,
              prodpath='spe_and_lc', group_min=25)

nu_obs.barycorr(infiles=lclist, prodpath=prodpath_ave, barytime='no')


### sum light curves from modules A and B, correct times of arrival with orbital parameters if necessary

In [ ]:
#sum light curves and make orbital correction
nu_obs.lcmath(infiles=lclist,
              outfile=prodpath_ave+'AB_sr.lc', prodpath=prodpath_ave, cmd_name='lcmath_orig',rewrite=True)

nu_obs.lcmath(infiles=[x+'_bary' for x in lclist],
              outfile=prodpath_ave+'AB_sr.lc_bary', prodpath=prodpath_ave, cmd_name='lcmath_bary', rewrite=True)


In [ ]:

#first you launch lcmath and then apply orbital correction
nu_obs.orb_correction_lc(
    filename=prodpath_ave+'AB_sr.lc_bary', prodpath=prodpath_ave)


### search for orbital period and make pulse profile of the light curve 
 After you have found the period (as the maximum of chi2 value, or fitting efsearch results with gaussian, write the prriod for your ObsID into a dictionaty periods_val in init file of your pulsar)

In [ ]:
####### orbital period search and light curve folding #######

nu_obs.make_efsearch(prodpath_ave+'AB_sr.lc_bary_orb_corr',
                     prodpath=prodpath_ave, p0='9.823', dres = '0.001', nper='64', nphase=16, cmd_name='efsearch_AB', rewrite=True)


In [ ]:
period = periods_val[ObsID]
nu_obs.make_efold(filename = prodpath_ave+'AB_sr.lc_bary_orb_corr', prodpath = prodpath_ave, period = str(period), cmd_name='efold_AB', nphase='128')